# LA County campaign contributions

In [47]:
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rc
%matplotlib inline
import numpy as np
import geojson
import jenkspy
import altair as alt
import altair_latimes as lat
alt.renderers.enable('notebook')
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [48]:
# contributions = []
# for f in ['input/2005-2006.xlsx','input/2007-2008.xlsx','input/2008-2009.xlsx','input/2009-2010.xlsx','input/2010-2011.xlsx','input/2011-2012.xlsx','input/2012-2013.xlsx','input/2013-2014.xlsx','input/2014-2015.xlsx','input/2015-2016.xlsx','input/2016-2017.xlsx','input/2017-2018.xlsx','input/2018-2019.xlsx','input/2019-2020.xlsx']:
#     data = pd.read_excel(f, 'public_search_results.cfm')
#     data.index = [basename(f)] * len(data)
#     contributions.append(data)

# contributions = pd.concat(contributions, sort=False)
# contributions.to_excel("output/merged.xlsx")

In [49]:
contributions = pd.read_csv('input/contributions.csv',low_memory=False, dtype={'contributor_zip_code': 'str', 'contributor_zip_code_ext': 'str'})

contributions.contributor_address_2 = contributions.contributor_address_2.fillna('')
contributions.occupation = contributions.occupation.fillna('')
contributions.employer = contributions.employer.fillna('')
contributions.description = contributions.description.fillna('')
contributions.contributor_first_name = contributions.contributor_first_name.fillna('')
contributions.contributor_zip_code_ext = contributions.contributor_zip_code_ext.fillna('')

### Clean up the table

In [50]:
contributions.columns

Index(['last_name', 'first_name', 'committee_id', 'committee_name',
       'office_type', 'district_number', 'schedule', 'type', 'period_beg_date',
       'period_end_date', 'election_date', 'date', 'amount_rcvd', 'amount_pd',
       'description', 'contributor_first_name', 'contributor_last_name',
       'contributor_address', 'contributor_address_2', 'contributor_city',
       'contributor_state', 'contributor_zip_code', 'contributor_zip_code_ext',
       'occupation', 'employer', 'rec_id', 'cycle'],
      dtype='object')

In [96]:
contributions['period_beg_date'] = pd.to_datetime(contributions['period_beg_date'])
contributions['period_end_date'] = pd.to_datetime(contributions['period_end_date'])
contributions['date'] = pd.to_datetime(contributions['date'])
contributions['year'], contributions['month'] = contributions['date'].dt.year, contributions['date'].dt.month
contributions['year'] = contributions['year'].astype(str)
contributions['month'] = contributions['month'].astype(str)
contributions['amount_rcvd'] = contributions['amount_rcvd'].astype(int)
contributions['monthyear'] = pd.to_datetime(contributions['date']).map(lambda dt: dt.replace(day=1))
contributions['candidate'] = contributions['first_name'] + ' ' + contributions['last_name']
contributions['candidate'] = contributions['candidate'].str.title()
contributions['candidate'] = contributions['candidate'].str.replace("Holly J. Mitchell", "Holly Mitchell")
contributions['type'] = contributions['type'].str.replace("Small Contributor Committee", "Small Donor Committee")
contributions['last_name'] = contributions['last_name'].str.upper()
contributions['committee_name'] = contributions['committee_name'].str.upper()
contributions['first_name'] = contributions['first_name'].str.upper()
contributions['months'] = contributions['date'].dt.strftime('%m/%Y').astype(str)
contributions["nu_contributor_address"] = contributions["contributor_address"].str.split('#').str[0].str.split(', ').str[0].str.split('Ste').str[0].str.split('Suite').str[0].str.split(',').str[0]

In [97]:
contributions['full_address'] =  contributions["nu_contributor_address"] + ', ' + contributions["contributor_city"] + ', ' + contributions["contributor_state"] + ', ' + contributions["contributor_zip_code"].astype(str)

In [98]:
contributions.head()
contributions.columns

Index(['last_name', 'first_name', 'committee_id', 'committee_name',
       'office_type', 'district_number', 'schedule', 'type', 'period_beg_date',
       'period_end_date', 'election_date', 'date', 'amount_rcvd', 'amount_pd',
       'description', 'contributor_first_name', 'contributor_last_name',
       'contributor_address', 'contributor_address_2', 'contributor_city',
       'contributor_state', 'contributor_zip_code', 'contributor_zip_code_ext',
       'occupation', 'employer', 'rec_id', 'cycle', 'year', 'month',
       'monthyear', 'candidate', 'months', 'contributor_apt_ste',
       'full_address', 'nu_contributor_address'],
      dtype='object')

In [99]:
geocoding = contributions.full_address.unique()

In [100]:
geocoding = pd.DataFrame(geocoding)

In [101]:
geocoding.to_csv('/Users/mhustiles/Desktop/github/notebooks/campaign-finance/output/addresses.csv')